# Preparation Steps

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import sys
import tempfile

from sklearn import svm, metrics
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import requests
from requests.auth import HTTPBasicAuth
from random import randint,random
import json
import os
import time

In [ ]:
AMBASSADOR_API_IP="localhost:8002"

def rest_request(deploymentName,request):
    payload = {'json': json.dumps(request)}
    response = requests.post(
                "http://"+AMBASSADOR_API_IP+"/seldon/"+deploymentName+"/api/v0.1/predictions",
                #"http://localhost:8009/predict",
                #"http://localhost:5300/predict",
                json=request)
                #data=payload)
    print(response)
    return response.json()   
    
def rest_request_auth(deploymentName,data,username,password):
    payload = {"data":{"ndarray":data.tolist()}}
    response = requests.post(
                "http://"+AMBASSADOR_API_IP+"/seldon/"+deploymentName+"/api/v0.1/predictions",
                json=payload,
                auth=HTTPBasicAuth(username, password))
    print(response.status_code)
    return response.json()   

In [ ]:
data_dir = '/tmp/tensorflow/mnist/input_data'
mnist = input_data.read_data_sets(data_dir, one_hot=True)

In [ ]:
test_data=mnist.test.images
test_labels=np.array(np.where(mnist.test.labels==1))[1]

# Demo

### The goal for this demo is to show deployed Intelligent Elements are capable of reacting to choreographies changes.
### There are two Intelligent Elements deployed:
### - mnist-svm-1-0 
#### This IE exposes the MNIST model for predicting handwritten digits. Whenever the choreography is created this IE will assume the role of producer.
### - mnistconsumer-1-1-0
#### This IE doesn't expose any machine model. It just returns what it receives. Whenever the choreography is created this IE will assume the role of consumer.


### In the first test we're going to run predictions without a choreography and verify messages

![title](img/without_choreography.png)

### In the second test we're going to run predictions with a choreography in place and verify messages

![title](img/with_choreography.png)

# Run Demo

## 1.- Run port-forwards

In [ ]:
os.popen('bash ambassador.sh')
time.sleep(5)

In [ ]:
os.popen('bash configuration.sh')
time.sleep(5)

## 2.- Reset test

In [ ]:
!curl -X DELETE --header 'Accept: application/json' 'http://localhost:8889/api/configuration/choreographies/ABC'

## 3.- Check existing choreographies

In [ ]:
!curl -X GET --header 'Accept: application/json' 'http://localhost:8889/api/configuration/choreographies/'

## 4.- Run prediction and verify consumer doesn't get any message


In [ ]:
test_index = randint(0, test_data.shape[0]-1)
plt.imshow(test_data[test_index].reshape((28,28))); plt.show()
test_samples = test_data[test_index].reshape((1,784)).tolist()
test_samples_with_ids = [{'id': '1', 'values':sample} for sample in test_samples]

request = {"data":{"ndarray":test_samples_with_ids}}

predictions = rest_request("mnist-svm-1-0",request)

print(json.dumps(predictions, indent=2))


## 5.- Configure choreography

In [ ]:
!curl -X POST --header 'Content-Type: application/json' --header 'Accept: application/json' -d '  { \
    "choreography_id": "ABC", \
    "channels": [ \
      { \
        "model_id": "mnistconsumer", \
        "model_version": "1", \
        "feature_collection_module": "", \
        "queue_forwardings": [], \
        "external_forwardings": [], \
        "feature_collection_triggers": [ \
          { \
            "queue_id": "1", \
            "topic": "MNIST" \
          } \
        ] \
      }, \
      { \
        "model_id": "mnist", \
        "model_version": "svm", \
        "feature_collection_module": "", \
        "queue_forwardings": [ \
          { \
            "queue_id": "1", \
            "topic": "MNIST" \
          } \
        ], \
        "external_forwardings": [], \
        "feature_collection_triggers": [] \
      } \
    ] \
  }' 'http://localhost:8889/api/configuration/choreographies/'

In [ ]:
!curl -X GET --header 'Accept: application/json' 'http://localhost:8889/api/configuration/choreographies/'

## 6.- Run prediction and verify messages arrive to consumer

In [ ]:
test_index = randint(0, test_data.shape[0]-1)
plt.imshow(test_data[test_index].reshape((28,28))); plt.show()
test_samples = test_data[test_index].reshape((1,784)).tolist()
test_samples_with_ids = [{'id': '1', 'values':sample} for sample in test_samples]

request = {"data":{"ndarray":test_samples_with_ids}}

predictions = rest_request("mnist-svm-1-0",request)

print(json.dumps(predictions, indent=2))
